# Robust Interval Guarantee for Quantum Measurements on Approximate States

This notebook illustrates the usage of the robustness application within Tequila following the paper ["Toward Reliability in the NISQ Era:  Robust Interval Guarantee for Quantum Measurements on Approximate States"](link) [1]. We will first briefly explain the results of the paper and the show how this can be applied to get robustness intervals for a variational quantum eigensolver (VQE).

On a high level, the goal of this paper is to derive robustness intervals for estimates of expectation values arising from imperfect approximations of a target state $\sigma$. These imperfections can arise from limited expressibility of ansatz states or from noise. To that end, three methods are proposed to obtain such robustness intervals based on different methods. As a crucial ingredient, each method expects the fidelity (or a corresponding lower bound) with the target state as input.

The first robustness interval takes only the first moment of the observable into account. Let $\rho$ denote the approximate state to which we have access to and which could be, for example, a noisy state obtained via the variational quantum eigensolver (VQE). Let $\sigma$ be the target state to which we do not have full access (e.g. the ground state of a Hamiltonian). For an observable $A$ which satisifies $-\mathbb{1} \leq A \leq \mathbb{1}$, a robustness interval for the (unknown) expectation value $\langle A\rangle_\sigma$ is given by

$$
    (1 - 2\epsilon)\cdot\langle A\rangle_\rho - 2\sqrt{\epsilon(1 - \epsilon)(1 - \langle A \rangle_\rho^2)} \ \leq \ \langle A\rangle_\sigma \ \leq \ (1 - 2\epsilon)\cdot\langle A\rangle_\rho + 2\sqrt{\epsilon(1 - \epsilon)(1 - \langle A \rangle_\rho^2)} 
$$

where $\epsilon \geq 0$ is such that the fidelity is lower bounded by $\mathcal{F}(\rho,\,\sigma) \geq 1-\epsilon$. The upper bound holds for small enough $\epsilon$ with $\epsilon \leq \frac{1}{2}(1 - \langle A\rangle_\rho)$ and the lower bound hold for $\epsilon$ with $\epsilon \leq \frac{1}{2}(1 + \langle A\rangle_\rho)$

The second method yields a lower bound to expectation values of positive observables $A\geq 0$ and takes into account second moments. It is obtained via the non-negativity of Gram matrices and is given by

$$
    \langle A\rangle_\sigma \geq (1 - 2\epsilon)\langle A\rangle_\rho -2 \sqrt{\epsilon(1-\epsilon)}\Delta A_\rho + \frac{\epsilon\langle A^2\rangle_\rho}{\langle A\rangle_\rho}.
$$

where $\epsilon$ is again a lower bound to the infidelity and $(\Delta A_\rho) = \langle A^2 \rangle_\rho - \langle A \rangle_\rho^2$ is the variance.

The third method is similar the second one and is also based on the non-negativity of Gram matrices. The difference here however is that the target state is assumed to be an eigenstste of a general observable $A$. Writing the target state as $\sigma = \lvert \psi \rangle\langle \psi\rvert$ with eigenstate $A\lvert \psi \rangle = \lambda \lvert \psi \rangle$, we have bounds on the eignvalue $\lambda$ in terms of the approximate state $\rho$:

$$
    \langle A\rangle_\rho - \Delta A_\rho\sqrt{\frac{\epsilon}{1-\epsilon}} \ \leq \ \lambda \ \leq \ \langle A\rangle_\rho + \Delta A_\rho\sqrt{\frac{\epsilon}{1-\epsilon}}
$$


In the following, we explain the implementation of these bounds in Tequila using two examples.

In [ ]:
import tequila as tq
import numpy as np
from tequila.apps.robustness import robustness_interval

## Example 1: VQE with imperfect ansatz

Here we use an Ansatz which cannot represent the eigenstate of a given Hamiltonian. Namely, we use the separable pair ansatz (SPA, see [arXiv:2105.03836](https://arxiv.org/abs/2105.03836)) to determine the ground state energies of BeH$_2$(4,8) within the basis-set free VQE ([arXiv:2008.02819](https://arxiv.org/abs/2008.02819)). We first setup the molecule.

In [ ]:
datadir="data/basis-set-free-molecules"
geometry = "Li .0 .0 .0\nH .0 .0 5.25"
mol = tq.Molecule(name=datadir + '/lih_5.25', geometry=geometry, n_pno=None)
H = mol.make_hamiltonian()

For reference, we compute the ground state and corresponding energy classically using numpy:

In [ ]:
eigenvalues, eigenstates = np.linalg.eigh(H.to_matrix())
ground_state_energy = eigenvalues[0]
ground_state = eigenstates[:,0]

Now, we create the ansatz circuit and optimize it to minimize the expectation value $\langle H\rangle_\theta$

In [ ]:
U = mol.make_upccgsd_ansatz('spa')
E = tq.ExpectationValue(U=U, H=H)
result = tq.minimize(E, silent=True)
print(f'VQE energy\t: {result.energy:.5f}')
print(f'exact energy\t: {ground_state_energy:.5f}')

To get the robustness interval, we need the fidelity of the optimized state and the target state, i.e. the true ground state. In practice, this needs to be estimated and a lower bound can be used as input. To illustrate the usage of the funcitonality, here we assume knowledge of the true fidelity which we compute in Tequila by encoding the target state as a Hamiltonian, and calculate its expectation value under the approximate (i.e. ansatz) state.

In [ ]:
ground_state_wfn = tq.QubitWaveFunction.from_array(ground_state)
ground_state_projector = tq.paulis.Projector(wfn=ground_state_wfn)
fidelity = tq.simulate(objective=tq.ExpectationValue(U=U, H=ground_state_projector), variables=result.variables, 
                       backend='qulacs')
print(f'ground state fidelity\t: {fidelity}')

Getting the robustness interval is now straight forward using the function $\texttt{robustness_interval}$. As input, we provide the function with the ansatz circuit $U$, the Hamiltonian $H$, the ground state fidelity and the optimal parameters. Setting the additional parameter $\texttt{kind="eigval"}$ indicates that the target state is an eigenstate of the Hamiltonian. In this setting, the function uses the Gramian eigenvalue method to compute the interval which is generally more accurate than the expectation value bounds. Finally, the function returns a tuple consisting of (lower_bound, vqe_energy, upper_bound).

In [ ]:
(lower_bound, vqe_energy, upper_bound), _ = robustness_interval(U=U, H=H, fidelity=fidelity, backend='qulacs', variables=result.variables,
                                                               kind='eigval')
print(f'upper bound\t: {upper_bound:.5f}')
print(f'VQE         \t: {vqe_energy:.5f}')
print(f'exact energy\t: {ground_state_energy:.5f}')
print(f'lower bound\t: {lower_bound:.5f}')

## Example 2: Noisy VQE

In [ ]:
from tequila.circuit.noise import BitFlip, DepolarizingError

We first setup the _H$_2$_ molecule in the STO-3G basis and print the resulting molecule together with the FCI energy:

In [ ]:
geometry = 'H .0 .0 .0\nH .0 .0 0.75'
mol = tq.Molecule(geometry='H .0 .0 .0\nH .0 .0 0.75', basis_set='sto-3g')
print(mol)
print(f"FCI energy\t: {mol.compute_energy('fci')}")

Now we create the Hamiltonian, intialize an UpCCGSD ansatz and the objective which is later optimized to approximate the corresponding ground state energy of _H$_2$_:

In [ ]:
H = mol.make_hamiltonian()
U = mol.make_upccgsd_ansatz()
E = tq.ExpectationValue(U=U, H=H)

print(f'Hamiltonian     \t: {H}')
print(f'number of terms \t: {len(H)}')
print(f'number of qubits\t: {H.n_qubits}')

We can calculate the exact ground state energy and eigenstate with numpy.

In [ ]:
eigvals, eigvecs = np.linalg.eigh(H.to_matrix())
exact_energy = min(eigvals)
exact_ground_state = eigvecs[:, 0]
print(f"exact ground state energy \t: {exact_energy:.6f}")
print(f"exact ground state        \t: {exact_ground_state}")

We now use a VQE to approximate the ground state and the corresponding energy. As real devices are subject to noise, we model this noise with a combination of bit flip errors acting on single qubit gates, and depolarizing noise acting on two qubit gates. We use 1% error probability for both noise models.

To setup this noise model in Tequila, we can simply run the following code line (see also the tutorial on noise [here](https://github.com/aspuru-guzik-group/tequila-tutorials/blob/main/Noise_tutorial.ipynb)):

In [ ]:
noise_model = BitFlip(p=0.01, level=1) + DepolarizingError(p=0.01, level=2)

And now, to run the noisy VQE optimization, we pass the noise model as argument to $\texttt{tq.minimize}$ and set the number of samples to 512.

In [ ]:
num_samples = 512

In [ ]:
# run noiseless VQE
result_noiseless = tq.minimize(E, backend='qiskit', silent=True)
error_noiseless = exact_energy - result_noiseless.energy
print(f'VQE energy without noise\t: {result_noiseless.energy:.6f}\t(error = {error_noiseless})')

# run noisy VQE
result_noisy = tq.minimize(E, backend='qiskit', samples=num_samples, noise=noise_model, silent=True)
error_noisy = exact_energy - result_noisy.energy
print(f'VQE energy with noise   \t: {result_noisy.energy:.6f}\t(error = {error_noisy})')

We see that the noiseless VQE is very close to the true energy. On the other hand, if there is noise, then the error of VQE is increased dramatically leading to an inaccurate energy estimate. We can also compute and compare the fidelity with the true ground state, leading to a similar picture.

In [ ]:
# encode exact wave function as a Hamiltonian
exact_ground_state_wfn = tq.QubitWaveFunction.from_array(exact_ground_state)
exact_ground_state_projector = tq.paulis.Projector(wfn=exact_ground_state_wfn)
objective = tq.ExpectationValue(U=U, H=exact_ground_state_projector)

# ground state fidelity of the noiseless VQE
fidelity_noiseless = tq.simulate(objective=objective, 
                                 variables=result_noiseless.variables, 
                                 backend='qiskit')

print(f'ground state fideltiy for the noiseless VQE\t: {fidelity_noiseless}')

# ground state fidelity of the noisy VQE
fidelity_noisy = tq.simulate(objective=objective, 
                             variables=result_noisy.variables, 
                             backend='qiskit',  
                             noise=noise_model,
                             samples=num_samples)

print(f'ground state fideltiy for the noisy VQE   \t: {fidelity_noisy}')

Given the ground state fidelity, we can now compute a robustness interval based on the noisy VQE. This interval is guaranteed to contain the true ground state energy. Note that, due to a finite number of samples, we need to repeat the computation a few times and caculate confidence intervals for the true value of the bounds (i.e. robustness interval).

In [ ]:
from scipy import stats

# compute intervals
intervals = []
variables = result_noisy.variables
for _ in range(30):
    f = tq.simulate(objective=objective, variables=variables,  backend='qiskit', noise=noise_model, 
                    samples=num_samples)
    interval, _ = robustness_interval(U=U, H=H, fidelity=f, kind='eigval', method='gramian',  noise=noise_model, 
                                      samples=num_samples, backend='qiskit', variables=result_noisy.variables)
    intervals.append(interval)

# compute mean energy
vqe_energy = np.mean([interval[1] for interval in intervals])

# compute confidence intervals
lower_bound_mean = np.mean([interval[0] for interval in intervals])
lower_bound_var = np.var([interval[0] for interval in intervals], ddof=1)
lower_bound = lower_bound_mean - np.sqrt(lower_bound_var / 30) * stats.t.ppf(q=0.99, df=29)

upper_bound_mean = np.mean([interval[2] for interval in intervals])
upper_bound_var = np.var([interval[2] for interval in intervals], ddof=1)
upper_bound = upper_bound_mean + np.sqrt(upper_bound_var / 30) * stats.t.ppf(q=0.99, df=29)
    
print(f'VQE energy               : {vqe_energy:.6f}')
print(f'True ground state energy : {exact_energy:.6f}')
print(f'Robustness interval      : [{lower_bound:.6f}, {upper_bound:.6f}]')

## References
[1] Maurice Weber, Abhinav Anand, Alba Cervera-Lierta, Jakob S. Kottmann, Thi Ha Kyaw, Bo Li, Alán Aspuru-Guzik, Ce Zhang and Zhikuan Zhao. "Toward Reliability in the NISQ Era: Robust Interval Guarantee for Quantum Measurements on Approximate States", [arxiv:2110.09793](https://arxiv.org/abs/2110.09793) (2021)